# Fremtidsrettet data ved hjelp av API
### Oppgave 2 - Datainnsamling

I denne delen har gruppen valgt å benytte API-er, nærmere bestemt et API med JSON-format, som hovedmetode for datainnhenting. Værdataene hentes fra [YR sin API-tjeneste](https://api.met.no/weatherapi/locationforecast/2.0/documentation), ved hjelp av geografiske koordinater innhentet fra [Google Maps](https://www.google.com/maps). Dette gjør det mulig å hente ut spesifikke værdata for valgte lokasjoner. Datasettet viser værprognoser for utvalgte steder for den kommende uken, og inkluderer blant annet informasjon om temperatur, vindhastighet og fuktighet. Nedenfor har gruppen gitt koordinater til noen steder, som kan brukes for å hente ut værdata. Funksjonen "make_watherJSON()" lager en JSON fil med hensyn på de gitte koordinater. 

In [3]:
import pandas as pd
import requests

In [4]:
#breddegrad og lengdegrad er hentet fra Google Maps
locations = [
    ("Stryn", 61.904275, 6.715641),
    ("Paris", 48.854289, 2.342042),
    ("London", 51.501814, -0.140605),
    ("Cape Town", -33.922348, 18.424031),
    ("New York", 40.710676, -74.006219)
]

#lager en oversiktstabell
data = []
for name, lat, lon in locations:
    data.append([name, lat, lon])

df = pd.DataFrame(data, columns=["Plass", "Breddegrad", "Lengdegrad"])
df.head()


,Plass,Breddegrad,Lengdegrad
0,Stryn,61.904275,6.715641
1,Paris,48.854289,2.342042
2,London,51.501814,-0.140605
3,Cape Town,-33.922348,18.424031
4,New York,40.710676,-74.006219


In [ ]:
#koden lager en JSON fil ut i fra koordinatene som blir puttet inn i linken 
def make_weatherJSON(lat, lon):
    url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={lat}&lon={lon}"
    headers = {'User-Agent': 'my-weather-app/1.0'}  #nødvendig for Yr API
    response = requests.get(url, headers=headers)
    
    
    if response.status_code != 200:
        raise Exception(f"API call failed with status code {response.status_code}")
    

    data = response.json()
    return data

#make_weatherJSON(61.904275, 6.715641)


### Oppgave 3 - Databehandlling

Funksjonen "clean_weather_data(lat, lon)" brukes til å identifisere hull i datasettene fra funksjonen "make_watherJSON(lat, lon)". Dersom noen hull blir identifisert, fyller funksjonen hullene med gjennomsnittet fra de tidligere målingene. Funksjonen "check_NaN_counter(lat, lon)" brukes som en kontroll, for å sjekke om det er noen hull igjen i datasettet. Dersom det ikke er noen hull, returneres 0. Til slutt blir daten presentert i funksjonen "get_temperatures_24(lat, lon)". Ved hjelp av list comprehensions, printes temperaturene for hver time i 24 timer.

In [ ]:
#rydder manglende verdier med fillna()
def clean_weather_data(lat, lon):
    data = make_weatherJSON(lat, lon)
    timeseries = data["properties"]["timeseries"]

    lst = []
    for entry in timeseries:
        time = entry["time"]
        details = entry["data"]["instant"]["details"]
        lst.append({
            "Time": time,
            "Temperature": details.get("air_temperature"),
            "Rain": entry["data"].get("next_1_hours", {}).get("details", {}).get("precipitation_amount"),
            "Wind speed": details.get("wind_speed")
        })

    
    df = pd.DataFrame(lst)


    #gjør om "Time" til datetime
    df["Time"] = pd.to_datetime(df["Time"], errors="coerce")


    #fyller inn evt. hull (NaN) med gjennomsnitt
    df.fillna(df.mean(numeric_only=True), inplace=True)
    
    return df


clean_weather_data(61.904275, 6.715641)

,Time,Temperature,Rain,Wind speed
0,2025-03-27 12:00:00+00:00,7.8,0.30,1.1
1,2025-03-27 13:00:00+00:00,7.5,0.20,0.8
2,2025-03-27 14:00:00+00:00,7.7,0.00,0.8
3,2025-03-27 15:00:00+00:00,7.6,0.00,0.7
4,2025-03-27 16:00:00+00:00,7.5,0.10,0.5
...,...,...,...,...
80,2025-04-05 06:00:00+00:00,2.7,0.55,1.5
81,2025-04-05 12:00:00+00:00,5.2,0.55,3.4
82,2025-04-05 18:00:00+00:00,3.0,0.55,1.8
83,2025-04-06 00:00:00+00:00,2.0,0.55,1.0


In [ ]:
#sjekker om det er noen hull i datasettet etterpå
def check_NaN_counter(lat, lon):
    data = clean_weather_data(lat, lon)
    missing_counts = df.isna().sum()
    
    print("Antall NaN-verdier per kolonne:")
    print(missing_counts)
    return missing_counts


check_NaN_counter(61.904275, 6.715641)

Antall NaN-verdier per kolonne:
Time           0
Temperature    0
Rain           0
Wind speed     0
dtype: int64


Time           0
Temperature    0
Rain           0
Wind speed     0
dtype: int64

In [ ]:
#henter ut temeraturer for de neste 24 timene
def get_temperatures_24(lat, lon):
    data = make_weatherJSON(lat, lon)
    timeseries = data["properties"]["timeseries"]
    
    temperatures = [
        (entry["time"], entry["data"]["instant"]["details"]["air_temperature"])
        for entry in timeseries[:24]
    ]
    
    return temperatures


#get_temperatures_24(61.904275, 6.715641)